In [1]:
import numpy as np
import scipy
import qiskit 
from mgbenchmark.main import mg_unitary_to_so, generate_jw_list, generate_jw_basis, unitary_to_superoperator, compound_matrix, mg_so_to_superoperator
from mgbenchmark.utils import generate_binary_strings, superop_to_dictionaries, dictionary_to_distribution, string_to_pauli, generate_jw_list_pauli, generate_jw_list
from qiskit import QuantumCircuit, Aer, execute
from qiskit.extensions import UnitaryGate
from qiskit.providers.aer import AerSimulator
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
from qiskit.tools.visualization import array_to_latex
import random

## Efficient Matchgate Tomography

We start off by generating a random $R \in SO(2n)$ matrix using the Haar Measure. We then calculate its quadratic hamiltonian elements

$h_{ij} = \frac{1}{4} \log(R)_{ij}$

and from that, find the quadratic hamiltonian:

$H = i \sum_{\mu \neq \nu} h_{\mu \nu} c_\mu c_\nu$

($c_\mu$ are the Clifford Algebra generators in the Jordan-Wigner basis). 

We then find the corresponding unitary:

$U = e^{-iH}$

In [259]:
n=8
jwlist = generate_jw_list(n)
S = scipy.stats.special_ortho_group.rvs(2*n)
log_x = scipy.linalg.logm(S) / 4

quadh = np.zeros((2**n, 2**n))
for i in range(len(log_x)):
    for j in range(len(log_x)):
        if i != j:
            quadh = quadh + ( jwlist[i] @ jwlist[j] ) * log_x[i][j] * 1j

U = scipy.linalg.expm(-1j * quadh)


Below is code modified from the matchgate benchmarking protocol, to estimate each matrix element 

$R_{ij} = \frac{1}{2^n} \text{Tr}(c_i^\dagger U c_j U^\dagger)$.

Each matrix element is estimated using $2 \ln (2 / \delta) / \epsilon^2$ shots, where $\epsilon$ is the precision and $\delta$ the failure probability.

This is carried out $4n^2$ times, giving a total number of shots of $8n^2 \lceil \ln (2 / \delta) / \epsilon^2 \rceil $.

In [262]:

# Specify key parameters:
epsilon = 0.05
delta = 0.01


# Generate the superoperator and the dictionaries

ugate = UnitaryGate(U, label="U")

shots = 0
jw_list = generate_jw_list_pauli(n)
str_list = generate_binary_strings(2*n, 1)


R = np.zeros((2*n, 2*n),dtype = 'complex128')

for x in range(2*n):
    for y in range(2*n):

        jobs_dict = {}
        circuit_dict = {}
        phase_dict = {}

        # Input iterated binary string 
        s_pair = str_list[x], str_list[y]

        # Process the input string into pauli + phase
        input_pauli = string_to_pauli(jw_list, s_pair[1])
        input_phase = input_pauli.phase
        input_pauli.phase = 0
        i_string = str(input_pauli)

        # Process the output string into pauli + phase
        output_pauli = string_to_pauli(jw_list, s_pair[0]).adjoint()
        output_phase = output_pauli.phase
        output_pauli.phase = 0
        j_string = str(output_pauli)

        phase = (-1j) ** (input_phase + output_phase)

        n_sample = int(np.ceil(2 * np.log(2 / delta) / ( epsilon ** 2)))

        tr = 0

        for _m in range(n_sample):

            # Create the quantum circuit
            qci = QuantumCircuit(n,n)
            lamb = 1
            qci_identifier = s_pair
            qci_key = ""

            # INITIALISE THE CIRCUIT IN PAULI EIGENSTATE
            # Apply the cliffords to the circuit according to the input. 
            # This prepares an eigenstate of the pauli & stores the eigenvalue as lamb
            # NOTE: The indices are reversed due to the Qiskit Convention, so q_(n-1) represents the first qubit and q_0 the last 
            for i in range(len(i_string)):
                prob = random.randint(0,1)
                if prob == 0:
                    qci_key += "0"
                else:
                    qci_key += "1"

                if i_string[i] == 'I':
                    if prob == 0:
                        qci.id(n-1-i)
                    else: 
                        qci.x(n-1-i)
                elif i_string[i] == 'Z':
                    if prob == 0:
                        qci.x(n-1-i)
                        lamb = lamb * -1
                    else:
                        qci.id(n-1-i)
                elif i_string[i] == 'X':
                    if prob == 0:
                        qci.h(n-1-i)
                    else:
                        qci.x(n-1-i)
                        qci.h(n-1-i)
                        lamb = lamb * -1
                elif i_string[i] == 'Y':
                    if prob == 0:
                        qci.h(n-1-i)
                        qci.s(n-1-i)
                    else:
                        qci.x(n-1-i)
                        qci.h(n-1-i)
                        qci.s(n-1-i)
                        lamb = lamb * -1

            # APPEND THE UNITARY GATE TO INPUT STATE
            qci.barrier()
            qci.append(ugate, list(range(n)))
            qci.barrier()

            # MEASURE THE OUTPUT IN PAULI BASIS
            # Apply the cliffords to the circuit according to the output string. This will give measurements in the pauli basis
            for j in range(len(j_string)):
                if j_string[j] == "I" or j_string[j] == "Z":
                    qci.id(n-1-j)
                elif j_string[j] == "X":
                    qci.h(n-1-j)
                elif j_string[j] == "Y":
                    qci.sdg(n-1-j)
                    qci.h(n-1-j)

            for j in range(len(j_string)):
                if j_string[j] == 'X' or j_string[j] == 'Y' or j_string[j] == 'Z':
                    qci.measure(n-1-j, n-1-j)

            qci_identifier += (qci_key,)

            if qci_identifier not in jobs_dict:
                jobs_dict[qci_identifier] = 1
                phase_dict[qci_identifier] = (lamb * phase)
                circuit_dict[qci_identifier] = qci
            elif qci_identifier in jobs_dict:
                jobs_dict[qci_identifier] += 1
            
            shots += 1

        for qci_identifier in jobs_dict:
            simulator = Aer.get_backend('aer_simulator')
            job = execute(circuit_dict[qci_identifier], backend=simulator, shots=jobs_dict[qci_identifier])
            result = job.result()
            counts = result.get_counts()
            for strings in counts:
                tr += (-1)**strings.count('1') * phase_dict[qci_identifier] * counts[strings]

        print(x, y, tr)
        R[x,y] = tr / n_sample




0 0 (-111+0j)
0 1 (299+0j)
0 2 (1145+0j)
0 3 (719+0j)
0 4 (-583+0j)
0 5 (-1149+0j)
0 6 (1741+0j)
0 7 (351+0j)
0 8 (-517+0j)
0 9 (1455+0j)
0 10 (841+0j)
0 11 (-859+0j)
0 12 (1343+0j)
0 13 (-791+0j)
0 14 (311+0j)
0 15 (2147+0j)
1 0 (-121+0j)
1 1 (381+0j)
1 2 (-1221+0j)
1 3 (1043+0j)
1 4 (1631+0j)
1 5 (457+0j)
1 6 (-37+0j)
1 7 (3015+0j)
1 8 (-713+0j)
1 9 (349+0j)
1 10 (487+0j)
1 11 (-517+0j)
1 12 (401+0j)
1 13 (845+0j)
1 14 (-951+0j)
1 15 (-167+0j)
2 0 (147+0j)
2 1 (1291+0j)
2 2 (-627+0j)
2 3 (209+0j)
2 4 (1403+0j)
2 5 (-535+0j)
2 6 (-179+0j)
2 7 (-817+0j)
2 8 (-1315+0j)
2 9 (387+0j)
2 10 (-1909+0j)
2 11 (331+0j)
2 12 (-1831+0j)
2 13 (189+0j)
2 14 (111+0j)
2 15 (2009+0j)
3 0 (-2713+0j)
3 1 (1243+0j)
3 2 (-313+0j)
3 3 (-25+0j)
3 4 (-93+0j)
3 5 (-155+0j)
3 6 (127+0j)
3 7 (663+0j)
3 8 (1457+0j)
3 9 (-1235+0j)
3 10 (-1357+0j)
3 11 (-295+0j)
3 12 (1257+0j)
3 13 (149+0j)
3 14 (1201+0j)
3 15 (137+0j)
4 0 (-1019+0j)
4 1 (499+0j)
4 2 (-1439+0j)
4 3 (-1195+0j)
4 4 (-1429+0j)
4 5 (771+0j)
4 6 (1097+

The code has been optimised to 'batch' circuit simulations together, to reduce the number of calls to the backend.

After getting an estimate $\tilde{R}$ of $R$, we repeat the previous calculations to find $\tilde{U}$, the estimated unitary:

In [263]:
jwlist = generate_jw_list(n)
log_x = scipy.linalg.logm(R) / 4

quadh = np.zeros((2**n, 2**n))
for i in range(len(log_x)):
    for j in range(len(log_x)):
        if i != j:
            quadh = quadh + ( jwlist[i] @ jwlist[j] ) * log_x[i][j] * 1j

U_calc = scipy.linalg.expm(-1j * quadh)

We now find the determinant of $R$, the $L2$ norm of $U - \tilde{U}$ and $R - \tilde{R}$, as well as the determinant norm $\sqrt{\det | \tilde{U} - U |}$.

In [274]:
np.linalg.det(R)

(0.9529342673428604+0j)

In [269]:
np.linalg.norm(U-U_calc, ord=2)

0.1403613541679722

In [270]:
np.linalg.norm(R-S, ord=2)

0.11638962485943251

In [268]:
np.sqrt(np.linalg.det(U-U_calc))

0j

The actual and estimated matrices $R$, $U$ are printed below:

In [271]:
array_to_latex(S)

<IPython.core.display.Latex object>

In [272]:
array_to_latex(R)

<IPython.core.display.Latex object>

In [273]:
array_to_latex(S- R)

<IPython.core.display.Latex object>

In [197]:
array_to_latex(U_calc)

<IPython.core.display.Latex object>

In [202]:
array_to_latex(U - U_calc)

<IPython.core.display.Latex object>